# Lapse-rate tropopause (LRT) is defined as the lowest level at which the lapse rate decreases to 2 deg K/km or less, provided also the average lapse rate between this level and all higher levels within 2 kilometers does not exceed 2 deg K/km

In [ ]:
import os
import json
import numpy as np
from scipy.interpolate import interp1d
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from warnings import filterwarnings
from awsgnssroutils.database import RODatabaseClient
from pprint import pprint
filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool` is a deprecated alias')


In [ ]:
axeslinewidth = 0.5 
plt.rcParams.update( {
  'font.family': "Times New Roman", 
  'font.size': 8,  
  'font.weight': "normal", 
  'text.usetex': True, 
  'xtick.major.width': axeslinewidth, 
  'xtick.minor.width': axeslinewidth, 
  'ytick.major.width': axeslinewidth, 
  'ytick.minor.width': axeslinewidth, 
  'axes.linewidth': axeslinewidth } ) 

In [ ]:
db = RODatabaseClient()
occs = db.query( missions="tsx", availablefiletypes="ucar_atmosphericRetrieval", 
                datetimerange=("2019-01-02","2019-01-03") )
print( occs )
files = occs.download( "ucar_atmosphericRetrieval" )
pprint( files )

In [ ]:
file = files[188]
d = Dataset( file, 'r' )
temperature = d.variables['temperature'][:]
pressure = d.variables['pressure'][:]
pressure = pressure * 0.01  # convert to hPa
alt = d.variables['altitude'][:]
alt = alt*1.e-3   # convert to km
#latitude = d.variables['refLatitude'].getValue()
d.close()

In [ ]:
hmax = 22
hmin = 6
dh = 0.1
h = np.arange(hmin,hmax,dh)
temf = interp1d(alt,temperature,'linear')
tem_utls = temf(h)
presf = interp1d(alt,pressure,'linear')
pres_utls = presf(h)
alt_utls = h 

# cold point tropopause
imin=np.argmin(tem_utls)
hcpt=alt_utls[imin]
tcpt=tem_utls[imin]
pcpt=pres_utls[imin]

print(hcpt,tcpt,pcpt)

In [ ]:
# lapse rate (WMO) tropopause
lapse=-np.diff(tem_utls)/dh
lapse2km=np.zeros(np.size(lapse))

n2km=int(2/dh)
nmax=len(lapse)
for i,z in enumerate(alt_utls[0:nmax-n2km]):
    lapse2km[i]=np.mean(lapse[i:i+n2km])
    #print(z,lapse[i],lapse2km[i])
    if (lapse[i] < 2 and lapse2km[i] < 2):
        #print(z,lapse[i],lapse2km[i])
        hlrt=alt_utls[i]
        tlrt=tem_utls[i]
        plrt=pres_utls[i]
        break

plt.clf()
plt.figure()
plt.plot(lapse,alt_utls[0:-1],lapse2km,alt_utls[0:-1])
plt.xlim( -1, 8)
plt.show()

In [ ]:
plt.clf()

plt.figure()
plt.plot(temperature,alt,'-',tcpt,hcpt,'o',tlrt,hlrt,'x')
plt.ylim( 6, 24)
plt.xlim(200, 240)
plt.show()

In [ ]:
occ_lat=[]
occ_lon=[]
hcpt=[]
tcpt=[]
pcpt=[]
hlrt=[]
tlrt=[]
plrt=[]

# range of heights to search for tropopause
hmax = 22
hmin = 6
dh = 0.1
h = np.arange(hmin,hmax,dh)

for k,file in enumerate(files):
    #print(occ)
    #file = path+'/'+occ
    d = Dataset( file, 'r' )
    temperature = d.variables['temperature'][:]
    pressure = d.variables['pressure'][:]
    pressure = pressure * 0.01  # convert to hPa
    alt = d.variables['altitude'][:]
    alt = alt*1.e-3   # convert to km
    occ_lat.append(d.variables['refLatitude'].getValue())
    occ_lon.append(d.variables['refLongitude'].getValue())
    d.close()
    
    if (min(alt) < hmin and max(alt) > hmax):
        temf = interp1d(alt,temperature,'linear')
        tem_utls = temf(h)
        presf = interp1d(alt,pressure,'linear')
        pres_utls = presf(h)
        alt_utls = h 
    
        imin=np.argmin(tem_utls)
        hcpt.append(alt_utls[imin])
        tcpt.append(tem_utls[imin])
        pcpt.append(pres_utls[imin])
        
    # lapse rate (WMO) tropopause
        lapse=-np.diff(tem_utls)/dh
        lapse2km=np.zeros(np.size(lapse))

        n2km=int(2.0/dh)
        nmax=len(lapse)
        for i,z in enumerate(alt_utls[0:nmax-n2km]):
            lapse2km[i]=np.mean(lapse[i:i+n2km])
            if (lapse[i] < 2 and lapse2km[i] < 2):
                hlrt.append(alt_utls[i])
                tlrt.append(tem_utls[i])
                plrt.append(pres_utls[i])
                break

    else:
        hcpt.append(-999.)
        tcpt.append(-999.)
        pcpt.append(-999.)
        
        hlrt.append(-999.)
        tlrt.append(-999.)
        plrt.append(-999.)

    

In [ ]:
plt.clf()

plt.figure()
plt.plot(occ_lat[:],hcpt[:],'o',occ_lat[:],hlrt[:],'x')
plt.ylim( 6, 24)
plt.xlim(-90,90)
plt.show()

In [ ]:
plt.clf()

plt.figure()
plt.plot(occ_lat[:],tcpt[:],'o',occ_lat[:],tlrt[:],'x')
plt.ylim(180, 240)
plt.xlim(-90,90)
plt.show()